In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import mmcv
from PIL import Image
from tqdm.auto import tqdm, trange
import os
import glob

In [3]:
def makedirs_ifno(paths):
    for path in paths:
        if os.path.exists(path):
            files = glob.glob(f'{path}*')
            for f in files:
                os.remove(f)
        else:
            os.makedirs(path)

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = torchvision.transforms.functional.to_pil_image(inp)
    plt.imshow(inp)
    plt.axis('off')
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [4]:
vid_file = 'videos/IMG_2139.MOV'
vid = mmcv.video.VideoReader(vid_file)

In [5]:
#from data import VideoFrameFolder
# data_transforms = {
#     'train': transforms.Compose([
#         #transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.ToTensor(),
#         #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }


# val_dataset = VideoFrameFolder('/home/tormod/ucph-erda-home/frames/',
#                               transform=data_transforms['val'])
# val_dataloader = torch.utils.data.DataLoader(val_dataset,
#                                             batch_size=30,
#                                             shuffle=False,
#                                             num_workers=8)

# MMDetection

In [6]:
#from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [7]:
# det_model = init_detector('mmpose_configs/faster_rcnn_r50_fpn_1x_coco.py',
#                           'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth')

In [8]:
# img =  vid[11*30]
# mmdet_results = inference_detector(det_model, img)
# show_result_pyplot(det_model, img, mmdet_results)
#mmdet_results

## MMTracking

In [9]:
import mmtrack
from mmtrack.apis import init_model, inference_vid, inference_mot

In [10]:
track_model = init_model('../mmtracking/configs/mot/tracktor/tracktor_faster-rcnn_r50_fpn_4e_mot17-private.py')
#track_model = init_model('../mmtracking/configs/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid.py')
#track_model = init_model('../mmtracking/configs/vid/dff/dff_faster_rcnn_r50_dc5_1x_imagenetvid.py')

2021-03-19 15:26:58,494 - mmtrack - INFO - load detector from: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth
2021-03-19 15:26:58,494 - mmtrack - INFO - Use load_from_http loader
2021-03-19 15:26:58,585 - mmtrack - INFO - load reid from: https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth
2021-03-19 15:26:58,585 - mmtrack - INFO - Use load_from_http loader


In [13]:
def bbox_crop(img, bbox):
    left, top, right, bot = bbox
    
    padding = (bot-top)*0.10
    left = left-padding
    top = top-padding
    right = right+padding
    bot = bot+padding
    
    height = bot-top+padding
    width = right-left+padding
    
    size = max(height, width)
    more_height = size - height
    more_width = size - width
    
    top = int(top - more_height/2)
    height = int(height + more_height)
    left = int(left - more_width/2)
    width = int(width + more_width)
    
    img = torchvision.transforms.functional.resized_crop(Image.fromarray(img), top, left, height, width, (500,500))
    return np.array(img)

def closest(true, guesses):
    return guesses[np.abs(guesses - true).sum(axis=1).argmin()]

def pick_track_result(prev_res, res):
    res = res['track_results'][0]
    res = res[:,1:-1]
    if res.shape[0] > 1:
        return closest(prev_res, res)
    else:
        return res[0]
    

In [19]:
makedirs_ifno(['tmp_frames/'])
bbox_res = np.zeros((1,4))
for frame_id in trange(vid.frame_cnt):
    img = vid[frame_id]
    track_results = inference_mot(track_model, img, frame_id)
    #new_frame = track_model.show_result(img, track_results['track_results'])
    left, top, right, bot =  pick_track_result(bbox_res[-1], track_results)
    #new_frame = bbox_crop(img, (left, top, right, bot))
    #mmcv.imwrite(new_frame, f'tmp_frames/{frame_id:06d}.png')
    
    bbox_res = np.concatenate((bbox_res, np.array([[left, top, right, bot]])))

bbox_res = bbox_res[1:] #discard initial zeros

    

KeyboardInterrupt: 

In [26]:
np.convolve(bbox_res[:,0], np.ones(5)/5, mode='same')

array([152.78586426, 204.24011841, 255.07853088, 255.24468079,
       253.52329102, 251.38067017, 249.22415771, 247.64385071,
       247.21447449, 247.58475037, 247.12400208, 245.5326355 ,
       242.62841187, 238.29132996, 232.9705658 , 228.05552673,
       224.38757629, 222.0401123 , 221.00371399, 220.90590515,
       221.94692993, 223.33516235, 224.20618896, 223.22239075,
       221.10664368, 217.54507751, 213.61348572, 209.90482788,
       207.75585022, 206.32749939, 205.6650177 , 204.99585876,
       204.84473267, 205.07672119, 205.65385437, 206.72636108,
       208.03677673, 209.34763489, 210.38744507, 211.57844543,
       212.68654175, 213.92669373, 215.39363403, 217.11373596,
       218.7460022 , 220.24671021, 221.55357971, 222.54416199,
       223.1963501 , 223.76142883, 224.59776306, 225.77688904,
       227.06315308, 228.81359863, 230.8035675 , 232.55115051,
       233.88486328, 235.08876953, 235.98603516, 236.43187866,
       236.73128967, 237.05582581, 237.50530701, 238.40

In [57]:
mmcv.frames2video('tmp_frames', 'zoom_test.mp4', fps=vid.fps, filename_tmpl='{:06d}.png')

FileNotFoundError: The start frame not found: tmp_frames/000000.png

# Torchvision detection

In [ ]:
# det = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
# val_dataset[0][0].shape

In [ ]:
# if 'pred' in globals():
#     del pred
# det.eval()
# pred = det(val_dataset[0][0].unsqueeze(0))
# #pred

In [ ]:
# def draw_best_box(frames, preds):
#     imgs = []
#     for f, p in zip(frames, preds): 
#         uint_inputframe = torchvision.transforms.functional.convert_image_dtype(f, torch.uint8)
#         wboxes = torchvision.utils.draw_bounding_boxes(uint_inputframe, p['boxes'][0:1], colors=['red'], width=3)
#         imgs.append(wboxes)
#     return torchvision.utils.make_grid(imgs)

# imshow(draw_best_box(val_dataset[0][0].unsqueeze(0), pred))